In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading the data

Since there is no header in the data itself, just adding random headers to not lose the first data.

This data serves no purpose so no need to store as pandas dataframe and we just turned it into a numpy array.

In [ ]:
X_train = pd.read_csv("/kaggle/input/data-science-london-scikit-learn/train.csv", names=list(range(40))).values
y_train = pd.read_csv("/kaggle/input/data-science-london-scikit-learn/trainLabels.csv", names=[0]).values.ravel()

X_test = pd.read_csv("/kaggle/input/data-science-london-scikit-learn/test.csv", names=list(range(40))).values

In [ ]:
X_train.shape, y_train.shape, X_test.shape

In [ ]:
np.unique(y_train, return_counts=True) # Balanced Data

# Plain Classification with Grid Search

In [ ]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
svm_params = {"C": [10, 100, 1000, 1e4, 1e5]}
nb_params = {"var_smoothing": [1e-8, 1e-9, 1e-10]}
knn_params = {"n_neighbors": range(5, 10)}
tree_params = {"max_depth":[5, 10, None], "min_samples_split":[2,3,4,5], "min_samples_leaf":[1,2,3,4]}

In [ ]:
svm_grid = GridSearchCV(SVC(), svm_params, n_jobs=-1, cv=10)
nb_grid = GridSearchCV(GaussianNB(), nb_params, n_jobs=-1, cv=10)
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, n_jobs=-1, cv=10)
tree_grid = GridSearchCV(DecisionTreeClassifier(), tree_params, n_jobs=-1, cv=10)

In [ ]:
svm_grid.fit(X_train, y_train)
nb_grid.fit(X_train, y_train)
knn_grid.fit(X_train, y_train)
tree_grid.fit(X_train, y_train);

In [ ]:
(
    svm_grid.best_score_, 
    nb_grid.best_score_,
    knn_grid.best_score_,
    tree_grid.best_score_
)

# Classification with Gaussian Mixture and Grid Search 

In [ ]:
from sklearn.mixture import GaussianMixture

X = np.r_[X_train, X_test]
X.shape

In [ ]:
lowest_bic = np.inf
bic = []
n_components_range = range(1,7)
best_gmm = None
for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components)
    gmm.fit(X)
    bic.append(gmm.aic(X))
    if bic[-1] < lowest_bic:
        lowest_bic = bic[-1]
        best_gmm = gmm
print(best_gmm)

best_gmm.fit(X)
gmm_train = best_gmm.predict_proba(X_train)
gmm_test = best_gmm.predict_proba(X_test)

In [ ]:
svm_grid.fit(gmm_train, y_train)
nb_grid.fit(gmm_train, y_train)
knn_grid.fit(gmm_train, y_train)
tree_grid.fit(gmm_train, y_train);

In [ ]:
(
    svm_grid.best_score_, 
    nb_grid.best_score_,
    knn_grid.best_score_,
    tree_grid.best_score_
)

In [ ]:
pred = knn_grid.predict(gmm_test)

In [ ]:
pred_df = pd.DataFrame(pred, columns=['Solution'], index=np.arange(1,9001))
pred_df.index.name = 'Id'
pred_df.reset_index(drop=False, inplace=True)

pred_df.head()

In [ ]:
pred_df.to_csv('submission.csv', index=False)